In [1]:
import pandas as pd
import numpy as np
import os
import glob
from PIL import Image
from facenet_pytorch import MTCNN, InceptionResnetV1
import torch
import torch.nn.functional as F
import math

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [3]:
resnet = InceptionResnetV1(pretrained='vggface2', device=device).eval()

In [4]:
array_of_embeddings = [[None for x in range(4)] for y in range(45)]

In [5]:
path = r'C:\Users\Vineet Kumar Verma\Desktop\Major Part - 1\Face Matching\training\*'
person = []
row_index = 0
for path in glob.glob(path):
    
    person.append(os.path.basename(path))
    person_img_path = glob.glob(os.path.join(path, '*.jpg'))
    
    col_index = 0
    array_of_embeddings[row_index][col_index] = os.path.basename(path)
    
    for img_path in person_img_path:
        
        img = Image.open(img_path)
        img = img.resize((160, 160), resample = Image.LANCZOS)
        img = np.array(img)
        mean = img.mean()
        stddev = img.std()
        adjusted_stddev = max(stddev, 1.0/math.sqrt(img.size))
        img = (img - mean)/adjusted_stddev
        img = torch.from_numpy(img).float().to(device)
        img = img.permute(2, 0, 1)
        img_embedding = resnet(img.unsqueeze(0))
        
        col_index = col_index + 1
        array_of_embeddings[row_index][col_index] = img_embedding.cpu().detach().numpy()
        
    row_index = row_index + 1

In [6]:
array_of_embeddings = np.array(array_of_embeddings)
embedding_df = pd.DataFrame(data = array_of_embeddings[:, 1:], index = array_of_embeddings[:, 0], columns = ['Aadhar', 'PAN', 'Photo'])

In [7]:
embedding_df.shape

(45, 3)

In [8]:
embedding_df.head()

,Aadhar,PAN,Photo
100,"[[0.03119058, -0.05560177, -0.030275773, 0.038...","[[-0.02418485, -0.07964193, 0.014518557, 0.040...","[[-0.039655436, -0.043306023, -0.03753889, 0.0..."
144,"[[-0.008161513, -0.03272914, 0.008121692, 0.02...","[[0.002646978, 0.02329809, 0.011009869, 0.0430...","[[-0.09759647, 0.017898235, -0.04802947, 0.003..."
145,"[[0.05043205, -0.026571298, -0.033735234, 0.02...","[[0.026021136, -0.035495855, -0.05510301, 0.05...","[[0.044944916, -0.00026974885, -0.06317894, 0...."
15,"[[-0.0068920534, -0.034521084, -0.07286961, 0....","[[0.061966754, -0.0049707447, -0.09104561, -0....","[[0.06369612, 0.005499538, -0.09948078, -0.004..."
150,"[[-0.035022575, 0.019373562, -0.02466429, -0.0...","[[-0.03116064, 0.04016489, -0.043046765, 0.076...","[[-0.014748882, -0.010233911, -0.026496585, -0..."


In [9]:
def calculate_euclidean_distance_between_pair_of_arrays_of_embeddings(df_index, arr1, arr2):
    
    len_arr = len(arr1)
    arr = [[None for x in range(len_arr)] for y in range(len_arr)]
    arr = np.array(arr)
    
    similarity_df = pd.DataFrame(data = arr[:,:], index = df_index, columns = df_index)
    
    row = 0
    for embed1 in arr1:
        
        embed1 = torch.from_numpy(embed1).float().to(device)
        col = 0
        for embed2 in arr2:
            
            embed2 = torch.from_numpy(embed2).float().to(device)
            similarity_df.iloc[row, col] = torch.dist(embed1, embed2, p=2).cpu().detach().numpy()
            col = col + 1
        
        row = row + 1
    
    return similarity_df

# def calculate_cosine_similarity_between_pair_of_arrays_of_embeddings(df_index, arr1, arr2):
    
#     len_arr = len(arr1)
#     arr = [[None for x in range(len_arr)] for y in range(len_arr)]
#     arr = np.array(arr)
    
#     similarity_df = pd.DataFrame(data = arr[:,:], index = df_index, columns = df_index)
    
#     row = 0
#     for embed1 in arr1:
        
#         embed1 = torch.from_numpy(embed1).float().to(device)
#         col = 0
#         for embed2 in arr2:
            
#             embed2 = torch.from_numpy(embed2).float().to(device)
#             similarity_df.iloc[row, col] = F.cosine_similarity(embed1, embed2).cpu().detach().numpy()
#             col = col + 1
        
#         row = row + 1
    
#     return similarity_df

# Aadhar and PAN

In [10]:
aadhar_pan_similarity_df = calculate_euclidean_distance_between_pair_of_arrays_of_embeddings(embedding_df.index,
                                                                                            embedding_df['Aadhar'],
                                                                                            embedding_df['PAN'])
aadhar_pan_similarity_df.head()
# aadhar_pan_similarity_df = calculate_cosine_similarity_between_pair_of_arrays_of_embeddings(embedding_df.index,
#                                                                                             embedding_df['Aadhar'],
#                                                                                             embedding_df['PAN'])
# aadhar_pan_similarity_df.head()

,100,144,145,15,150,165,173,185,190,211,...,65,720,73,743,744,750,787,791,84,86
100,1.1249468,1.248987,0.94985074,0.84347224,1.3608162,1.1700618,1.291453,1.3143312,1.1674507,1.3013529,...,1.2365026,1.2569273,1.3402457,1.2200788,1.0664337,1.3197284,1.2940716,1.3076136,1.2147554,1.0598155
144,1.2316425,1.0967193,1.2817563,1.3074806,1.2874907,1.0826404,1.2770717,1.2497509,1.1484106,1.340102,...,1.1313891,1.0188057,1.2368032,1.3418547,1.1348579,1.2717345,1.1570641,0.850904,1.2432611,1.0924494
145,1.3157766,1.1886638,1.0272193,1.0952785,1.1556824,1.0906973,1.200355,1.0404088,0.9399374,1.1722499,...,1.2319205,1.283053,1.2624198,1.2320844,1.1869135,1.143932,1.1324673,1.0964601,1.1363239,1.062724
15,1.5158911,1.378691,1.175729,1.1310198,1.141001,1.1986754,1.267084,1.3907672,1.2114846,1.3757821,...,1.105973,1.3657241,1.2617737,1.3283597,1.3299484,1.2060018,1.4163526,1.3563664,1.2517307,1.2361416
150,1.3732208,1.356593,1.3453794,1.2350374,1.2620649,1.3414484,1.2449733,1.3246169,1.3108959,1.3452805,...,1.2489277,1.2343371,1.2722667,1.0957359,1.287985,1.4522246,1.3680265,1.4135249,1.2413058,1.3637834


In [23]:
def extract_all_diag_elements_and_min_in_each_row_except_diag_element(df):
    
    min_of_each_row = []
    for i in df.index:
        arr = []
        for j in df.columns:
            if i != j:
                arr.append(float(df.loc[i, j]))
            else:
                continue
        
        min_of_each_row.append(min(arr))
    
    
    comparison_df = pd.DataFrame(index = df.index, 
                                 columns = ['distance between Aadhar and PAN of same person',
                                                             'min (distance between Aadhar and PAN of different person)'],
                                dtype = float)
    comparison_df['distance between Aadhar and PAN of same person'] = np.diag(df).astype(float)
    comparison_df['min (distance between Aadhar and PAN of different person)'] = min_of_each_row
    return comparison_df

# def extract_all_diag_elements_and_max_in_each_row_except_diag_element(df):
    
#     max_of_each_row = []
#     for i in df.index:
#         arr = []
#         for j in df.columns:
#             if i != j:
#                 arr.append(float(df.loc[i, j]))
#             else:
#                 continue
        
#         max_of_each_row.append(max(arr))
    
    
#     comparison_df = pd.DataFrame(index = df.index, 
#                                  columns = ['similarity between Aadhar and PAN of same person',
#                                                              'max (similarity between Aadhar and PAN of different person)'],
#                                 dtype = float)
#     comparison_df['similarity between Aadhar and PAN of same person'] = np.diag(df).astype(float)
#     comparison_df['max (similarity between Aadhar and PAN of different person)'] = max_of_each_row
#     return comparison_df

In [24]:
comparison_aadhar_pan = extract_all_diag_elements_and_min_in_each_row_except_diag_element(aadhar_pan_similarity_df)
comparison_aadhar_pan.head()

,distance between Aadhar and PAN of same person,min (distance between Aadhar and PAN of different person)
100,1.124947,0.843472
144,1.096719,0.850904
145,1.027219,0.924286
15,1.131020,0.890996
150,1.262065,1.095736


In [15]:
def distance_between_same_smaller_than_min_distance_between_different(df):
    
    df['is first column smaller than second'] = np.where(df[df.columns[0]] < df[df.columns[1]], 1, 0)
    return df
# def similarity_between_same_greater_than_max_similarity_between_different(df):
    
#     df['is first column greater than second'] = np.where(df[df.columns[0]] > df[df.columns[1]], 1, 0)
#     return df

In [17]:
comparison_aadhar_pan = distance_between_same_smaller_than_min_distance_between_different(comparison_aadhar_pan)
comparison_aadhar_pan.head()
# comparison_aadhar_pan = similarity_between_same_greater_than_max_similarity_between_different(comparison_aadhar_pan)
# comparison_aadhar_pan.head()

,distance between Aadhar and PAN of same person,min (distance between Aadhar and PAN of different person),is first column smaller than second
100,1.124947,0.843472,0
144,1.096719,0.850904,0
145,1.027219,0.924286,0
15,1.131020,0.890996,0
150,1.262065,1.095736,0


In [18]:
sum(comparison_aadhar_pan['is first column smaller than second'])

19

In [19]:
threshold = min(comparison_aadhar_pan['min (distance between Aadhar and PAN of different person)'])
print(threshold)

0.8121740221977234


In [16]:
# sum(comparison_aadhar_pan['is first column greater than second'])

19

In [24]:
# threshold = max(comparison_aadhar_pan['max (similarity between Aadhar and PAN of different person)'])
# print(threshold)

0.6701867580413818

In [20]:
df = comparison_aadhar_pan[comparison_aadhar_pan['distance between Aadhar and PAN of same person'] < threshold]
df
# df = comparison_aadhar_pan[comparison_aadhar_pan['similarity between Aadhar and PAN of same person'] > threshold]
# df

,distance between Aadhar and PAN of same person,min (distance between Aadhar and PAN of different person),is first column smaller than second
165,0.772063,0.846471,1
185,0.750572,0.870885,1
485,0.784386,0.887989,1
49,0.704490,0.992259,1
65,0.752317,1.126868,1
750,0.599075,0.965258,1
787,0.662829,0.845253,1


In [21]:
print(len(df))

7


# Aadhar and Photo

In [22]:
aadhar_photo_similarity_df = calculate_euclidean_distance_between_pair_of_arrays_of_embeddings(embedding_df.index,
                                                                                            embedding_df['Aadhar'],
                                                                                            embedding_df['Photo'])
aadhar_photo_similarity_df.head()

# aadhar_photo_similarity_df = calculate_cosine_similarity_between_pair_of_arrays_of_embeddings(embedding_df.index,
#                                                                                             embedding_df['Aadhar'],
#                                                                                             embedding_df['Photo'])
# aadhar_photo_similarity_df.head()

,100,144,145,15,150,165,173,185,190,211,...,65,720,73,743,744,750,787,791,84,86
100,0.91485703,1.2021034,1.0179911,1.1097921,1.4052042,1.163284,1.140204,1.2568513,1.2751178,1.2736351,...,1.0953131,1.232333,1.0956341,1.1487391,1.0986116,1.2080889,1.3567904,1.2304091,1.0512516,1.2837853
144,1.2724625,1.092839,1.1537526,1.3560839,1.1771886,1.1187584,1.1214759,1.1755092,1.1795686,0.9603163,...,1.1580482,1.0754218,1.0304811,1.1629915,1.2256925,1.3158242,1.1212168,1.0219917,1.2302092,1.1072783
145,1.176971,1.3681135,1.2615552,1.1790622,1.2165803,1.1730325,1.1606154,1.0432692,1.1924478,1.3556929,...,1.159918,1.2709211,1.2252059,1.2886244,1.1273504,1.0678614,1.1643205,1.2535347,1.3930523,1.1509892
15,1.3775862,1.376927,1.3111717,1.1997974,1.2764235,1.2899804,1.2672957,1.3026861,1.2697232,1.408266,...,1.1046067,1.2413747,1.393201,1.260099,1.2763995,1.0499874,1.3852725,1.4091543,1.3473874,1.269157
150,1.2355752,1.338948,1.4975508,1.3399997,0.73234266,1.356994,1.336731,1.3000373,1.264677,1.1194081,...,1.4194111,1.3277858,1.297026,1.1195873,1.2846385,1.3824956,1.4757919,1.4411343,1.3065466,1.3906877


In [25]:
def extract_all_diag_elements_and_min_in_each_row_except_diag_element(df):
    
    min_of_each_row = []
    for i in df.index:
        arr = []
        for j in df.columns:
            if i != j:
                arr.append(float(df.loc[i, j]))
            else:
                continue
        
        min_of_each_row.append(min(arr))
    
    
    comparison_df = pd.DataFrame(index = df.index, 
                                 columns = ['distance between Aadhar and Photo of same person',
                                                             'min (distance between Aadhar and Photo of different person)'],
                                dtype = float)
    comparison_df['distance between Aadhar and Photo of same person'] = np.diag(df).astype(float)
    comparison_df['min (distance between Aadhar and Photo of different person)'] = min_of_each_row
    return comparison_df

# def extract_all_diag_elements_and_max_in_each_row_except_diag_element(df):
    
#     max_of_each_row = []
#     for i in df.index:
#         arr = []
#         for j in df.columns:
#             if i != j:
#                 arr.append(float(df.loc[i, j]))
#             else:
#                 continue
        
#         max_of_each_row.append(max(arr))
    
    
#     comparison_df = pd.DataFrame(index = df.index, 
#                                  columns = ['similarity between Aadhar and Photo of same person',
#                                                              'max (similarity between Aadhar and Photo of different person)'],
#                                 dtype = float)
#     comparison_df['similarity between Aadhar and Photo of same person'] = np.diag(df).astype(float)
#     comparison_df['max (similarity between Aadhar and Photo of different person)'] = max_of_each_row
#     return comparison_df

In [26]:
comparison_aadhar_photo = extract_all_diag_elements_and_min_in_each_row_except_diag_element(aadhar_photo_similarity_df)
comparison_aadhar_photo.head()

# comparison_aadhar_photo = extract_all_diag_elements_and_max_in_each_row_except_diag_element(aadhar_photo_similarity_df)
# comparison_aadhar_photo.head()

,distance between Aadhar and Photo of same person,min (distance between Aadhar and Photo of different person)
100,0.914857,1.017991
144,1.092839,0.960316
145,1.261555,1.034587
15,1.199797,1.049987
150,0.732343,1.119408


In [28]:
comparison_aadhar_photo = distance_between_same_smaller_than_min_distance_between_different(comparison_aadhar_photo)
comparison_aadhar_photo.head()

,distance between Aadhar and Photo of same person,min (distance between Aadhar and Photo of different person),is first column smaller than second
100,0.914857,1.017991,1
144,1.092839,0.960316,0
145,1.261555,1.034587,0
15,1.199797,1.049987,0
150,0.732343,1.119408,1


In [29]:
sum(comparison_aadhar_photo['is first column smaller than second'])

# sum(comparison_aadhar_photo['is first column greater than second'])

26

In [30]:
threshold = min(comparison_aadhar_photo['min (distance between Aadhar and Photo of different person)'])
print(threshold)

0.7951412200927734


In [31]:
df = comparison_aadhar_photo[comparison_aadhar_photo['distance between Aadhar and Photo of same person'] < threshold]
df
# df = comparison_aadhar_photo[comparison_aadhar_photo['similarity between Aadhar and Photo of same person'] > threshold]
# df

,distance between Aadhar and Photo of same person,min (distance between Aadhar and Photo of different person),is first column smaller than second
150,0.732343,1.119408,1
190,0.765195,0.969868,1
307,0.740370,0.994297,1
46,0.569595,0.846861,1
485,0.774231,0.883273,1
49,0.695636,0.993089,1
52,0.689138,0.946663,1
53,0.745430,1.047329,1
561,0.729858,0.857481,1
720,0.690333,0.882333,1


In [32]:
print(len(df))

13


# PAN and Photo

In [252]:
pan_photo_similarity_df = calculate_cosine_similarity_between_pair_of_arrays_of_embeddings(embedding_df.index,
                                                                                            embedding_df['PAN'],
                                                                                            embedding_df['Photo'])
pan_photo_similarity_df.head()

,100,144,145,15,150,165,173,185,190,211,...,65,720,73,743,744,750,787,791,84,86
100,[0.569613],[0.27929842],[0.213435],[-0.10264892],[0.09440001],[0.37023443],[0.30492043],[0.12542081],[0.1538906],[0.23548794],...,[0.1313123],[0.10629165],[0.6086922],[0.21119383],[0.16927986],[0.07288602],[0.35815018],[0.07461136],[0.27356315],[0.09969843]
144,[0.36276692],[0.51299727],[0.13197444],[0.11649351],[0.079539575],[0.5194673],[0.4134995],[0.32026476],[0.44446704],[0.40920627],...,[0.4790185],[0.08924302],[0.3832845],[0.24001649],[0.15434952],[0.32470518],[0.30354834],[0.19241668],[0.48403192],[0.25752702]
145,[0.42853174],[0.46340334],[0.43685198],[0.3559432],[0.20909199],[0.43763173],[0.58343786],[0.2715553],[0.23182525],[0.12174097],...,[0.55097806],[0.16975811],[0.4862316],[0.31983557],[0.31954858],[0.43028364],[0.30805904],[0.22735928],[0.41623145],[0.1571015]
15,[0.30944437],[0.34450495],[0.4107715],[0.695203],[0.08017993],[0.1826342],[0.37281823],[0.3687735],[0.28368488],[0.13823254],...,[0.5549339],[0.31223783],[0.2736179],[0.38940477],[0.47435126],[0.40479156],[0.10889851],[0.2765097],[0.34792462],[0.23647627]
150,[0.09186682],[0.13952886],[0.08645578],[0.24397604],[0.33966324],[0.3082679],[0.31609368],[0.25309438],[0.08623936],[0.30237162],...,[0.40457547],[0.20488282],[0.17849432],[0.35847026],[0.09600626],[0.08098245],[0.17281052],[0.16622713],[-0.004645936],[0.25762713]


In [253]:
def extract_all_diag_elements_and_max_in_each_row_except_diag_element(df):
    
    max_of_each_row = []
    for i in df.index:
        arr = []
        for j in df.columns:
            if i != j:
                arr.append(float(df.loc[i, j]))
            else:
                continue
        
        max_of_each_row.append(max(arr))
    
    
    comparison_df = pd.DataFrame(index = df.index, 
                                 columns = ['similarity between PAN and Photo of same person',
                                                             'max (similarity between PAN and Photo of different person)'],
                                dtype = float)
    comparison_df['similarity between PAN and Photo of same person'] = np.diag(df).astype(float)
    comparison_df['max (similarity between PAN and Photo of different person)'] = max_of_each_row
    return comparison_df

In [254]:
comparison_pan_photo = extract_all_diag_elements_and_max_in_each_row_except_diag_element(pan_photo_similarity_df)
comparison_pan_photo.head()

,similarity between PAN and Photo of same person,max (similarity between PAN and Photo of different person)
100,0.569613,0.608692
144,0.512997,0.533941
145,0.436852,0.583438
15,0.695203,0.554934
150,0.339663,0.491139


In [255]:
comparison_pan_photo = similarity_between_same_greater_than_max_similarity_between_different(comparison_pan_photo)
comparison_pan_photo.head()

,similarity between PAN and Photo of same person,max (similarity between PAN and Photo of different person),is first column greater than second
100,0.569613,0.608692,0
144,0.512997,0.533941,0
145,0.436852,0.583438,0
15,0.695203,0.554934,1
150,0.339663,0.491139,0


In [257]:
sum(comparison_pan_photo['is first column greater than second'])

16

In [259]:
threshold = max(comparison_pan_photo['max (similarity between PAN and Photo of different person)'])
print(threshold)

0.7088738679885864


In [260]:
df = comparison_pan_photo[comparison_pan_photo['similarity between PAN and Photo of same person'] > threshold]
df

,similarity between PAN and Photo of same person,max (similarity between PAN and Photo of different person),is first column greater than second
165,0.714757,0.597296,1
300,0.778669,0.544762,1


In [261]:
print(len(df))

2


# Testing

In [33]:
array_of_embeddings = [[None for x in range(4)] for y in range(5)]

In [34]:
path = r'C:\Users\Vineet Kumar Verma\Desktop\Major Part - 1\Face Matching\testing\*'
person = []
row_index = 0
for path in glob.glob(path):
    
    person.append(os.path.basename(path))
    person_img_path = glob.glob(os.path.join(path, '*.jpg'))
    
    col_index = 0
    array_of_embeddings[row_index][col_index] = os.path.basename(path)
    
    for img_path in person_img_path:
        
        img = Image.open(img_path)
        img = img.resize((160, 160), resample = Image.LANCZOS)
        img = np.array(img)
        mean = img.mean()
        stddev = img.std()
        adjusted_stddev = max(stddev, 1.0/math.sqrt(img.size))
        img = (img - mean)/adjusted_stddev
        img = torch.from_numpy(img).float().to(device)
        img = img.permute(2, 0, 1)
        img_embedding = resnet(img.unsqueeze(0))
        
        col_index = col_index + 1
        array_of_embeddings[row_index][col_index] = img_embedding.cpu().detach().numpy()
        
    row_index = row_index + 1

In [35]:
array_of_embeddings = np.array(array_of_embeddings)
embedding_df = pd.DataFrame(data = array_of_embeddings[:, 1:], index = array_of_embeddings[:, 0], columns = ['Aadhar', 'PAN', 'Photo'])

In [36]:
array_of_embeddings.shape

(5, 4)

In [37]:
embedding_df.head()

,Aadhar,PAN,Photo
87,"[[0.08032808, 0.0022187908, -0.0061137364, 0.0...","[[0.10873219, 0.036200453, -0.015876528, 0.009...","[[0.07251646, 0.025673952, -0.039972663, 0.039..."
91,"[[0.03421601, 0.009918489, -0.031607434, 0.075...","[[-0.04772474, -0.023173077, -0.031549525, 0.1...","[[0.0748417, 0.03776666, -0.060070746, 0.06341..."
95,"[[-0.010804167, -0.022445457, -0.06283371, -0....","[[0.0025419795, -0.041995212, -0.050703246, 0....","[[0.017880706, -0.029694777, -0.063536525, -0...."
97,"[[-0.0024183118, -0.07374744, -0.07514099, 0.0...","[[-0.051738735, -0.06656351, 0.019510383, 0.03...","[[0.0042888564, -0.0600945, -0.04075473, 0.033..."
99,"[[0.055735037, 0.0032483032, -0.004439715, -0....","[[-0.014526795, -0.071204685, -0.044314064, 0....","[[0.03232144, -0.0040156967, -0.023385758, 0.0..."


In [38]:
embedding_df.index

Index(['87', '91', '95', '97', '99'], dtype='object')

In [39]:
aadhar_pan_similarity_df = calculate_cosine_similarity_between_pair_of_arrays_of_embeddings(embedding_df.index,
                                                                                            embedding_df['Aadhar'],
                                                                                            embedding_df['PAN'])
aadhar_pan_similarity_df.head()

,87,91,95,97,99
87,[0.5937584],[0.3948523],[0.40398878],[0.1059763],[0.63193804]
91,[0.111164175],[0.21231918],[0.18072614],[0.08497542],[0.51137114]
95,[0.14769775],[0.17736691],[0.5647969],[0.25961915],[0.28175452]
97,[-0.00091993954],[0.104667366],[0.07080146],[0.36843252],[0.31343186]
99,[0.2736417],[-0.07032224],[0.17448653],[0.20737918],[0.4764104]


In [40]:
aadhar_photo_similarity_df = calculate_cosine_similarity_between_pair_of_arrays_of_embeddings(embedding_df.index,
                                                                                            embedding_df['Aadhar'],
                                                                                            embedding_df['Photo'])
aadhar_photo_similarity_df.head()

,87,91,95,97,99
87,[0.62562877],[0.31277004],[0.31414136],[0.1352819],[0.40975517]
91,[0.14450958],[0.4635543],[0.09590107],[0.24551731],[0.22435337]
95,[0.39672953],[0.3100075],[0.7683025],[0.28417668],[0.48357168]
97,[-0.025387287],[0.2863755],[0.3884441],[0.7905817],[0.42410618]
99,[0.22114114],[0.41484344],[0.36171725],[0.41350424],[0.59712976]


In [41]:
pan_photo_similarity_df = calculate_cosine_similarity_between_pair_of_arrays_of_embeddings(embedding_df.index,
                                                                                            embedding_df['PAN'],
                                                                                            embedding_df['Photo'])
pan_photo_similarity_df.head()

,87,91,95,97,99
87,[0.7724253],[0.23237859],[0.2803244],[0.07735313],[0.32673392]
91,[0.3615555],[0.07772973],[0.12222495],[0.11913731],[0.21912244]
95,[0.44129798],[0.36162055],[0.57703376],[0.17086561],[0.22607997]
97,[0.15932733],[0.08452206],[0.32180429],[0.5321894],[0.22604932]
99,[0.384317],[0.2840317],[0.47372523],[0.47274742],[0.451739]
